# Notebook for ActorCritic RL model for OpenAI LunarLander

In [385]:
#Python3
import numpy as np
import gym
import Box2D
import torch
import torchvision
import argparse
from time import time
from torch.autograd import Variable
from torch.distributions import Categorical
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


Give args

In [386]:
class args_parse():
    def __init__(self):
        self.num_episodes = 5000
        self.max_frames = 200
        self.gamma = 0.99
        self.lr = 0.99
        

Now we define the Actor Critic Class

In [387]:
class ActorCritic(nn.Module):
    """ Creates a functionally merged Actor and Critic network models, both
        built of fully connected layers
        -- input:
            - env_space: Size of input vector from environment(int)
            - action_space: Size of output vector for action softmax onehot
            - n_hidden (optional): Number nodes in hidden layers (int)
            - mode: Training or testing/demoing?
        -- output:
            - action: The softmax onehot encoded action vector
            - expected_value: The expected value from the state-action pair
    """
    def __init__(self, env_space, action_space, n_hidden=6, lr=args.lr):
        super(ActorCritic, self).__init__()
        self.action_space = action_space
        self.env_space = env_space
        self.n_hidden = n_hidden
        self.saved_actions = []
        self.saved_rewards = []
        
        #define combined Network
        self.l1 = nn.Linear(env_space, n_hidden*2)
        self.l2 = nn.Linear(n_hidden*2, n_hidden)
        self.action_head = nn.Linear(n_hidden, action_space)
        self.value_head = nn.Linear(n_hidden, 1)
        
    def forward(self, env):
        x = F.relu(self.l1(env))
        x = F.relu(self.l2(x))
        action_scores = self.action_head(x)
        state_value = self.value_head(x)
        return F.softmax(action_scores), state_value
    
    def select_action(self, env):
        action_scores, _ = self.forward(env)
        action = Categorical(action_scores).sample().data.numpy()[0]
        return action

In [388]:
def train(actor_critic):
    """ Trains a policy to take an action giving an environment observation,
        with the goal of maximizing the rewards from each trial
        -- input:
            - actor_critic: A combined net of class ActorCritic
        -- output:
            - actor_critic: A combined network trained over args.num_episodes
    """
    optimizer = optim.Adam(actor_critic.parameters(), lr=args.lr)
    for episode in range(0, args.num_episodes):
        samples = collect_samples(actor_critic)
        saved_actions = [data[1] for data in samples]
        expected_rewards = [data[2] for data in samples]
        R=0
        policy_losses = []
        value_losses = []
        rewards = []
        for data in samples[::-1]:
            R = data[3] + (args.gamma * R)
            rewards.insert(0,R)
        cum_rewards = sum(rewards)
        rewards = torch.Tensor(rewards)
        rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float32).eps)
        for soft_action, expected_reward, r in \
        zip(saved_actions, expected_rewards, rewards):
            reward = r - expected_reward.data[0]
            m = Categorical(soft_action)
            action = m.sample()
            log_prob = m.log_prob(action)
            policy_losses.append(-log_prob * reward)
            value_losses.append(F.smooth_l1_loss(expected_reward, Variable(torch.Tensor([r]))))
        optimizer.zero_grad()
        loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
        loss.backward()
        optimizer.step()
        print(cum_rewards)
        
        

In [389]:
def collect_samples(actor_critic):
    """ Collects samples from an environment episode either until terminal
        state or until max number of frames reached
        -- input:
            - actor_critic: A combined net of class ActorCritic
        -- output:
            - samples: A list of state, soft action, expected state value,
                       and an actual state-action rewards, all of class 
                       torch.autograd.Variable
    """
    env = gym.make('LunarLander-v2')
    state = Variable(torch.FloatTensor(env.reset()))
    isterminal = False
    samples = [None for x in range(0, args.max_frames)]
    frame = 0
    while (isterminal != True) or (frame < args.max_frames):
        soft_act, expected_reward = actor_critic(state)
        action_taken = actor_critic.select_action(state)
        next_state, actual_reward, isterminal, _ = env.step(action_taken)
        next_state = Variable(torch.FloatTensor(next_state))
        samples[frame] = [state,\
                          soft_act,\
                          expected_reward,\
                          actual_reward]
        state = next_state
        frame += 1
    samples =  [x for x in samples if x != None]
    return samples

In [390]:
actor_critic = ActorCritic(env_space=8, action_space=4)

In [391]:
args = args_parse()

In [392]:
trained_model = train(actor_critic)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-977201.9489603087
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/spencerlheureux/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


-945526.0000921355
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-903229.8970272511
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-698570.4508556537
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-988659.5360365564
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-905938.6561392527
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1031135.9819118625
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1024930.9964110213
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-979926.5226168123
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-919024.1470382062
WARN: gym.spaces.Box autodetected d

-908940.2032037091
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-989514.1691785904
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-925816.15634753
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1009412.002003188
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-712201.9907756019
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-886115.7508101226
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1038628.85155675
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1020152.2935462568
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-899195.0318816516
WARN: gym.spaces.Box autodetected dtype

-920866.0777916296
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1032195.346965129
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1013278.2173904076
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-913082.9018369142
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-920438.416967476
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1035223.2996110602
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1032488.8600681049
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-938807.8850295765
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1010002.3279165432
WARN: gym.spaces.Box autodetected 

...

-1010727.4065799215
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-937219.6103697589
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1026003.717379282
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-964575.9173063408
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-949035.2421571314
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1016432.8300098109
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-917712.0377252467
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-923149.3392176917
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-994962.1385911156
WARN: gym.spaces.Box autodetected d

-975612.771844798
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-884218.586229268
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1005582.9810546164
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1027705.7007756799
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-985944.9557439368
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-788599.6362143117
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-948624.6886909304
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-951861.7154999687
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-950764.0339140086
WARN: gym.spaces.Box autodetected dty

-939599.245365821
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1037237.8258753198
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-961706.7026870621
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-933843.4586296119
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-980337.8505886924
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-934302.5683847291
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-975247.4810518303
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-963183.4973422547
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1019962.3789986717
WARN: gym.spaces.Box autodetected dt

-962957.9409399836
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1012466.0623053694
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-954199.8813515874
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-961478.906687561
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-891178.9829485565
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-897174.9085112559
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-908525.7222979006
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1018152.9408023679
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-937238.4873101956
WARN: gym.spaces.Box autodetected dt

-996832.5460871989
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1050688.966219471
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1033361.8739093011
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-918841.7710792162
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1036236.1878579813
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1012979.602495884
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-920466.3495034359
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-892113.6737080954
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-1041712.4589262712
WARN: gym.spaces.Box autodetected 